
# 🧠 Apuntes de MongoDB (Basado en el curso "Introducción a las Bases de Datos NoSQL")

MongoDB es una **base de datos NoSQL orientada a documentos**, que almacena información en **documentos BSON (Binary JSON)**.
Estos documentos son similares a los objetos JSON, lo que permite **flexibilidad en el esquema** y **alta escalabilidad horizontal**.

## Características principales
- No requiere esquema fijo.  
- Escala horizontalmente mediante *sharding*.  
- Permite indexación, agregaciones y consultas complejas.  
- Ideal para datos semiestructurados o heterogéneos.



## 2. Problemática de ejemplo: Sistema de Cotizaciones “RabloQuote”

La empresa **RabloQuote** guarda solicitudes de cotización de clientes.  
Cada cotización puede tener:

- ID del cliente (`id_rablo`)
- Productos solicitados
- Estatus del proceso
- Fechas de creación y respuesta


In [ ]:

# Ejemplo de documento JSON
{
  "id_rablo": "R123",
  "cliente": "Juan Pérez",
  "productos": [
    {"nombre": "Sensor de temperatura", "cantidad": 5, "precio_unitario": 25.5},
    {"nombre": "Motor DC", "cantidad": 2, "precio_unitario": 60.0}
  ],
  "estatus": "En proceso",
  "fecha_creacion": ISODate("2025-10-31T09:00:00Z"),
  "fecha_respuesta": null
}



## 3. Creación y Conexión a MongoDB

### En el shell de MongoDB


In [ ]:

mongosh                                 # Activar mongo shell en consola

use miBaseDeDatos;                      # Mover a base de datos
db.createCollection("cotizaciones");    # Crea nueva colección

db                         # Base de datos actual
show dbs                   # Listar bases de datos
show collections           # Listar colecciones
db.cotizaciones.stats()    # Detalles de colección


### Inserción de documentos de ejemplo

In [ ]:

# Insertar varios documentos en coleccion

db.cotizaciones.insertMany([
  {
    id_rablo: "R123",
    cliente: "Juan Pérez",
    productos: [
      { nombre: "Sensor de temperatura", cantidad: 5, precio_unitario: 25.5 },
      { nombre: "Motor DC", cantidad: 2, precio_unitario: 60.0 }
    ],
    estatus: "En proceso",
    fecha_creacion: new Date("2025-10-31T09:00:00Z"),
    fecha_respuesta: null
  },
  {
    id_rablo: "R123",
    cliente: "Juan Pérez",
    productos: [
      { nombre: "Arduino Uno", cantidad: 1, precio_unitario: 45.0 }
    ],
    estatus: "Finalizado",
    fecha_creacion: new Date("2025-10-25T10:00:00Z"),
    fecha_respuesta: new Date("2025-10-27T15:00:00Z")
  },
  {
    id_rablo: "R987",
    cliente: "María Gómez",
    productos: [
      { nombre: "Sensor de luz", cantidad: 10, precio_unitario: 5.0 }
    ],
    estatus: "Pendiente",
    fecha_creacion: new Date("2025-10-30T11:00:00Z"),
    fecha_respuesta: null
  }
]);

db.cotizaciones.find()  # Ver contenido actualizado en coleccion      



## 4. Conexión desde Python (con PyMongo)


In [12]:

from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27017/")
db = client["miBaseDeDatos"]
collection = db["cotizaciones"]

for doc in collection.find():
    print(doc)


{'_id': ObjectId('6904f9ea2e3790a884cebea4'), 'id_rablo': 'R123', 'cliente': 'Juan Pérez', 'productos': [{'nombre': 'Sensor de temperatura', 'cantidad': 5, 'precio_unitario': 25.5}, {'nombre': 'Motor DC', 'cantidad': 2, 'precio_unitario': 60}], 'estatus': 'En proceso', 'fecha_creacion': datetime.datetime(2025, 10, 31, 9, 0), 'fecha_respuesta': None}
{'_id': ObjectId('6904f9ea2e3790a884cebea5'), 'id_rablo': 'R123', 'cliente': 'Juan Pérez', 'productos': [{'nombre': 'Arduino Uno', 'cantidad': 1, 'precio_unitario': 45}], 'estatus': 'Finalizado', 'fecha_creacion': datetime.datetime(2025, 10, 25, 10, 0), 'fecha_respuesta': datetime.datetime(2025, 10, 27, 15, 0)}
{'_id': ObjectId('6904f9ea2e3790a884cebea6'), 'id_rablo': 'R987', 'cliente': 'María Gómez', 'productos': [{'nombre': 'Sensor de luz', 'cantidad': 10, 'precio_unitario': 5}], 'estatus': 'Pendiente', 'fecha_creacion': datetime.datetime(2025, 10, 30, 11, 0), 'fecha_respuesta': None}



## 5. Tipos de Consultas en MongoDB


### Consultas Básicas:
SHELL

In [ ]:
db.cotizaciones.find();
db.cotizaciones.findOne({ id_rablo: "R123" });
db.cotizaciones.find({ estatus: "Pendiente" });
db.cotizaciones.find({ "productos.nombre": "Arduino Uno" });


PYTHON

In [ ]:
collection.find()
collection.find_one({ "id_rablo": "R123" })
collection.find({ "estatus": "Pendiente" })
collection.find({ "productos.nombre": "Arduino Uno" })

# Ejemplo:

# Filtrar por estatus
print("\nDocumentos con estatus 'Pendiente':")

for doc in collection.find({"estatus": "Pendiente"}):
    print(doc)



Documentos con estatus 'Pendiente':
{'_id': ObjectId('6904f9ea2e3790a884cebea6'), 'id_rablo': 'R987', 'cliente': 'María Gómez', 'productos': [{'nombre': 'Sensor de luz', 'cantidad': 10, 'precio_unitario': 5}], 'estatus': 'Pendiente', 'fecha_creacion': datetime.datetime(2025, 10, 30, 11, 0), 'fecha_respuesta': None}


### Consultas con operadores:
SHELL

In [ ]:

db.cotizaciones.find({
  fecha_creacion: { $lt: new Date("2025-10-25T00:00:00Z") }
});

db.cotizaciones.find({
  $or: [{ estatus: "Pendiente" }, { estatus: "En proceso" }]
});


PYTHON

In [ ]:
from datetime import datetime

# $lt significa "less than" 
for doc in collection.find({"fecha_creacion": {"$lt": datetime(2025, 10, 25, 0, 0, 0)}}):
    print(doc)


query = {
    "$or": [
        {"estatus": "Pendiente"},
        {"estatus": "En proceso"}
    ]
}

for doc in collection.find(query):
    print(doc)


{'_id': ObjectId('6904f9ea2e3790a884cebea4'), 'id_rablo': 'R123', 'cliente': 'Juan Pérez', 'productos': [{'nombre': 'Sensor de temperatura', 'cantidad': 5, 'precio_unitario': 25.5}, {'nombre': 'Motor DC', 'cantidad': 2, 'precio_unitario': 60}], 'estatus': 'En proceso', 'fecha_creacion': datetime.datetime(2025, 10, 31, 9, 0), 'fecha_respuesta': None}
{'_id': ObjectId('6904f9ea2e3790a884cebea6'), 'id_rablo': 'R987', 'cliente': 'María Gómez', 'productos': [{'nombre': 'Sensor de luz', 'cantidad': 10, 'precio_unitario': 5}], 'estatus': 'Pendiente', 'fecha_creacion': datetime.datetime(2025, 10, 30, 11, 0), 'fecha_respuesta': None}


### Proyección:
SHELL

In [ ]:

# Proyección en SHELL
db.cotizaciones.find(
  { id_rablo: "R123" },
  { cliente: 1, estatus: 1, fecha_creacion: 1, _id: 0 }
);

# Ordenamiento en SHELL
db.cotizaciones.find().sort({ fecha_creacion: -1 }).limit(3);


PYTHON

In [25]:
# Proyeccion y ordenamiento en PYTHON

# Filtrar documentos con id_rablo = "R123"
query = {"id_rablo": "R123"}

# Proyección: seleccionar únicamente los campos deseados
# 1 = incluir, 0 = excluir
projection = {"cliente": 1, "estatus": 1, "fecha_creacion": 1, "_id": 0}

for doc in collection.find(query, projection):
    print(doc)

# Ordenar los documentos por fecha_creacion descendente (-1)
# y tomar solo los primeros 3 documentos
for doc in collection.find().sort("fecha_creacion", -1).limit(3):
    print(doc)

{'cliente': 'Juan Pérez', 'estatus': 'En proceso', 'fecha_creacion': datetime.datetime(2025, 10, 31, 9, 0)}
{'cliente': 'Juan Pérez', 'estatus': 'Finalizado', 'fecha_creacion': datetime.datetime(2025, 10, 25, 10, 0)}
{'_id': ObjectId('6904f9ea2e3790a884cebea4'), 'id_rablo': 'R123', 'cliente': 'Juan Pérez', 'productos': [{'nombre': 'Sensor de temperatura', 'cantidad': 5, 'precio_unitario': 25.5}, {'nombre': 'Motor DC', 'cantidad': 2, 'precio_unitario': 60}], 'estatus': 'En proceso', 'fecha_creacion': datetime.datetime(2025, 10, 31, 9, 0), 'fecha_respuesta': None}
{'_id': ObjectId('6904f9ea2e3790a884cebea6'), 'id_rablo': 'R987', 'cliente': 'María Gómez', 'productos': [{'nombre': 'Sensor de luz', 'cantidad': 10, 'precio_unitario': 5}], 'estatus': 'Pendiente', 'fecha_creacion': datetime.datetime(2025, 10, 30, 11, 0), 'fecha_respuesta': None}
{'_id': ObjectId('6904f9ea2e3790a884cebea5'), 'id_rablo': 'R123', 'cliente': 'Juan Pérez', 'productos': [{'nombre': 'Arduino Uno', 'cantidad': 1, 'pre

## 6. Actualización y Eliminación

In [ ]:

# Actualizar estatus
db.cotizaciones.updateOne(
  { id_rablo: "R987" },
  { $set: { estatus: "Finalizado", fecha_respuesta: new Date() } }
);

# Agregar producto
db.cotizaciones.updateOne(
  { id_rablo: "R123", estatus: "En proceso" },
  { $push: { productos: { nombre: "Relay", cantidad: 4, precio_unitario: 10 } } }
);

# Eliminar
db.cotizaciones.deleteOne({ estatus: "Finalizado" });


## 7. Agregaciones

In [ ]:

# Total de cotizaciones por cliente
db.cotizaciones.aggregate([
  { $group: { _id: "$id_rablo", total_cotizaciones: { $sum: 1 } } }
]);


In [ ]:

# Promedio de precio por producto
db.cotizaciones.aggregate([
  { $unwind: "$productos" },
  { $group: { _id: "$productos.nombre", promedio_precio: { $avg: "$productos.precio_unitario" } } },
  { $sort: { promedio_precio: -1 } }
]);


In [ ]:

# Valor total por cotización
db.cotizaciones.aggregate([
  { $unwind: "$productos" },
  {
    $group: {
      _id: "$_id",
      cliente: { $first: "$cliente" },
      total: { $sum: { $multiply: ["$productos.cantidad", "$productos.precio_unitario"] } }
    }
  },
  { $sort: { total: -1 } }
]);



## 8. Resumen: MongoDB en el Desarrollo Real 💡

| Etapa | Descripción | Ejemplo |
|--------|--------------|----------|
| Modelado flexible | MongoDB permite adaptar la estructura de los datos sin redefinir esquemas | Cotizaciones con campos opcionales por cliente |
| Inserción ágil | Ideal para datos que llegan desde APIs, logs o IoT | Guardar solicitudes en tiempo real |
| Consultas dinámicas | Filtros por estado, cliente o fechas | `find({estatus:"En proceso"})` |
| Agregaciones analíticas | Procesamiento de métricas sin exportar a SQL | Calcular totales o promedios |
| Integración con Python / Node.js | Uso con PyMongo o Mongoose | Dashboards o sistemas internos |

**Conclusión:** MongoDB ofrece un enfoque documental, flexible y escalable para manejar datos complejos o no estructurados.  
El framework de agregación lo convierte en una herramienta poderosa para analítica interna sin necesidad de SQL.



## 9 Valor agregado sobre SQL

| Característica         | MongoDB (NoSQL)                                                                 | SQL (Relacional)                                           | Ventaja MongoDB                                                        |
|------------------------|---------------------------------------------------------------------------------|------------------------------------------------------------|------------------------------------------------------------------------|
| Esquema flexible       | Documentos JSON/BSON                                                             | Tablas rígidas                                             | Se pueden agregar campos o estructuras anidadas sin migraciones.      |
| Procesamiento de arrays| `$unwind` permite expandir arrays                                               | Requiere `JOIN` o subqueries                               | Más natural y rápido para estructuras jerárquicas.                     |
| Pipeline               | Permite múltiples etapas (filtrar, proyectar, agrupar, ordenar) en un solo flujo | SQL necesita combinar `WHERE`, `GROUP BY`, `HAVING`, subqueries | Más legible y eficiente para análisis complejos.                       |
| Datos jerárquicos      | Los documentos pueden contener arrays y objetos anidados                        | Normalmente se normaliza en múltiples tablas              | Se reduce la necesidad de múltiples joins, mejora la velocidad.       |
| Escalabilidad horizontal | Fácil de shardear y distribuir                                                  | Más complejo                                               | Ideal para grandes volúmenes de datos.                                 |

---

### ✅ Conclusión

Las agregaciones en MongoDB:

- Permiten **analizar y resumir grandes volúmenes de datos directamente en la base**, evitando traerlos a la aplicación.  
- Manejan **estructuras jerárquicas y arrays de forma natural**, algo que en SQL requiere joins complicados.  
- Ofrecen un **flujo en pipeline muy flexible** para combinar filtrado, agrupamiento, proyección y ordenamiento.  
- Dan **rapidez y escalabilidad** en sistemas que manejan millones de documentos.  

En resumen, **MongoDB agrega valor** cuando necesitas análisis sobre datos semi-estructurados o jerárquicos, sin sacrificar rendimiento, algo que sería mucho más engorroso en SQL.